## **BIBLIOTECAS PARA EDA**

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import os

## **ACESSO AOS DADOS**

In [5]:
path = "../data/"

if os.path.exists(f"{path}train.csv"):
    df_train = pd.read_csv(f"{path}train.csv")
    df_test = pd.read_csv(f"{path}test.csv")
    print("Dados carregados")
else:
    print("Erro: Arquivos de dados não encontrados")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Treino: (227845, 32)
Teste: (56962, 31)


,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,265519,161919.0,1.946747,-0.752526,-1.355130,-0.661630,1.502822,4.024933,-1.479661,1.139880,...,0.076197,0.297537,0.307915,0.690980,-0.350316,-0.388907,0.077641,-0.032248,7.32,0
1,180306,124477.0,2.035149,-0.048880,-3.058693,0.247945,2.943487,3.298697,-0.002192,0.674782,...,0.038628,0.228197,0.035542,0.707090,0.512885,-0.471198,0.002520,-0.069002,2.99,0
2,42665,41191.0,-0.991920,0.603193,0.711976,-0.992425,-0.825838,1.956261,-2.212603,-5.037523,...,-2.798352,0.109526,-0.436530,-0.932803,0.826684,0.913773,0.038049,0.185340,175.10,0
3,198724,132624.0,2.285718,-1.500239,-0.747565,-1.668119,-1.394143,-0.350339,-1.427984,0.010010,...,-0.139670,0.077013,0.208310,-0.538236,-0.278032,-0.162068,0.018045,-0.063005,6.10,0
4,82326,59359.0,-0.448747,-1.011440,0.115903,-3.454854,0.715771,-0.147490,0.504347,-0.113817,...,-0.243245,-0.173298,-0.006692,-1.362383,-0.292234,-0.144622,-0.032580,-0.064194,86.10,0


## **EDA**

### **Missing values**

In [ ]:
df_train.isnull().sum().sort_values(ascending=False)

Não foram identificados valores ausentes, indicando boa integridade estrutural do dataset.

### **Análise do `target`**

In [ ]:
# O quão desbalanceado é o problema?
fraud_percent = df_train['Class'].value_counts(normalize=True) * 100
fraud_counts = df_train['Class'].value_counts()

print(f"Números absolutos:\n {fraud_counts}")
print(f"\nNúmeros em porcentagem:\n {fraud_percent}")

A taxa de fraude representa 0,17% do total das transações, caracterizando um cenário de forte desbalanceamento. Esse comportamento é típico de problemas reais de detecção de fraude, nos quais eventos fraudulentos são raros.

Devido ao desbalanceamento, métricas como acurácia tornam-se inadequadas. Optou-se pela utilização da **ROC-AUC** e técnicas de balanceamento via `class_weight`.

### **Análise Univariada**

In [ ]:
# Em busca de variáveis potencialmente problemáticas
display(df_train.columns)
display(df_train.dtypes)

A variável `id` representa apenas um identificador único da transação, não contendo informação preditiva relevante. Sua inclusão poderia introduzir ruído ou padrões espúrios no modelo.

O dataset contém 28 variáveis numéricas anonimizadas `(V1–V28)`, resultantes de transformação por PCA. A ausência de semântica direta reforça a necessidade de abordagens orientadas à performance e interpretabilidade pós-modelagem.

A variável temporal representa aproximadamente dois dias consecutivos de transações, permitindo investigar possíveis padrões temporais de fraude. É preciso fazer maiores análises para descobrir se contém informação preditiva ou risco de vazamento atrelado a `Time`.



### **Variáveis Originais**

### **Time**

In [ ]:
print(df_train['Time'].describe().round(2))


A análise da variável temporal indica que eventos fraudulentos não são uniformemente distribuídos e apresentam concentração em intervalos específicos, sugerindo hipoteses de padrões sequenciais ou ataques agrupados. Entretanto, como o dataset representa o intervalo contínuo de aproximadamente dois dias, a inclusão dessa variável exige cuidado para evitar aprendizado espúrio de padrões temporais não generalizáveis.

In [ ]:
df_train.groupby(pd.qcut(df_train['Time'], 10))['Class'].mean()

A análise da taxa de fraude segmentada por decil temporal revelou variação significativa ao longo do período analisado, com diferenças de até 3,5 vezes entre os intervalos de menor e maior incidência. Observam-se picos específicos no início e no meio da janela temporal, sugerindo possível ocorrência de eventos concentrados ou comportamento episódico de fraude. Tal padrão indica que a variável temporal contém informação preditiva relevante, porém potencialmente sensível a efeitos de clusterização sequencial.

A variável `Time` será mantida por potencial de capturar padrões sequenciais de fraude. Experimentos comparativos foram conduzidos para avaliar seu impacto na generalização.

In [ ]:
plt.figure(figsize=(6,5))

sns.kdeplot(
    data=df_train[df_train['Class'] == 0],
    x='Time',
    label='Não Fraude',
    fill=True,
    alpha=0.4
)

sns.kdeplot(
    data=df_train[df_train['Class'] == 1],
    x='Time',
    label='Fraude',
    fill=True,
    alpha=0.4
)

plt.title('Distribuição de Time por Classe')
plt.xlabel('Time (segundos desde a primeira transação)')
plt.ylabel('Densidade')
plt.legend()
plt.show()


O gráfico revela alta sobreposição entre as distribuições de fraude e não fraude. Embora haja leve deslocamento nas médias, a dispersão e o formato das curvas são bastante semelhantes, indicando baixo poder discriminatório isolado. Assim, a variável `Time` pode conter informação contextual, mas não apresenta separabilidade estrutural significativa entre as classes.

### **Amount**

In [ ]:
plt.figure(figsize=(6,5))

sns.kdeplot(
    data=df_train[df_train['Class'] == 0],
    x=np.log1p(df_train[df_train['Class'] == 0]['Amount']),
    label='Não Fraude',
    fill=True,
    alpha=0.4
)

sns.kdeplot(
    data=df_train[df_train['Class'] == 1],
    x=np.log1p(df_train[df_train['Class'] == 1]['Amount']),
    label='Fraude',
    fill=True,
    alpha=0.4
)

plt.title('Distribuição Logarítmica de Amount por Classe')
plt.xlabel('log(Amount + 1)')
plt.ylabel('Densidade')
plt.legend()
plt.show()


Gráficos de variáveis como `Amount` tendem a ser muito assimétricos e de difícil visualização, por isso a decisão de utilizar escala log.

A variável `Amount` apresenta comportamento bimodal na classe fraudulenta, sugerindo a existência de dois padrões distintos de valor entre transações fraudulentas. Entretanto, observa-se formato de distribuição semelhante na classe não fraudulenta, com alta sobreposição entre as curvas. Dessa forma, embora haja diferença média entre as classes, o poder discriminatório isolado da variável é limitado.

In [ ]:
mean_diff = df_train.groupby('Class')['Amount'].mean()
std = df_train['Amount'].std()

print((mean_diff[1] - mean_diff[0]) / std)


A diferença padronizada entre as médias de `Amount` foi de aproximadamente 0.15 desvios-padrão, indicando efeito pequeno. Embora fraudes apresentem, em média, valores monetários maiores, a alta variabilidade da variável resulta em significativa sobreposição entre as distribuições.

### **Variáveis Transformadas**

In [ ]:
# As distribuições diferem entre fraude e não fraude?
corr = df_train.corr()['Class'].abs().sort_values(ascending=False)
top_v = corr.drop(['Class']).head(5).index.tolist()
top_v


Para identificar as variáveis com maior capacidade discriminatória entre fraude e não fraude, foi realizada uma análise comparativa das médias por classe.

Inicialmente, calculou-se a diferença absoluta entre as médias das variáveis para as classes 0 (não fraude) e 1 (fraude). Como as variáveis `V1–V28` representam componentes principais ortogonais e padronizadas, a magnitude do deslocamento médio entre classes reflete diretamente o grau de separação linear entre fraude e não fraude ao longo desses eixos.

In [ ]:
for col in top_v:
    plt.figure(figsize=(6,5))
    sns.kdeplot(data=df_train[df_train['Class']==0], x=col, label='Não Fraude', fill=True, alpha=0.4)
    sns.kdeplot(data=df_train[df_train['Class']==1], x=col, label='Fraude', fill=True, alpha=0.4)
    plt.title(f'Distribuição de {col} por Classe')
    plt.legend()
    plt.show()


A partir desse critério, observou-se que as variáveis `V17, V14, V12, V10` e `V3` apresentaram os maiores deslocamentos médios entre as classes, indicando forte separabilidade estatística no espaço transformado.

In [ ]:
plt.figure(figsize=(6,5))
sns.boxplot(data=df_train, x='Class', y='V3')
plt.title('Boxplot de V3 por Classe')
plt.show()


Selecionando uma variável qualquer dentre as variáveis de maior diferença das médias por classe para exemplo (`V3`), observa-se que a variável `V3` apresenta deslocamento significativo entre as classes. Enquanto transações legítimas concentram-se próximas de zero, transações fraudulentas apresentam distribuição deslocada para valores negativos. A diferença nas medianas e no intervalo interquartil indica forte poder discriminativo univariado.

O deslocamento consistente da mediana sugere que `V3` carrega informação estrutural relevante para separação das classes, corroborando os resultados obtidos na análise de diferenças médias.

### **Análise Bivariada**

In [ ]:
plt.figure(figsize=(6,5))

df_plot = df_train.sample(10000).copy()

df_plot['Class'] = df_plot['Class'].map({0: 'Não Fraude', 1: 'Fraude'})

sns.scatterplot(
    data=df_plot,
    x='V3',
    y='V14',
    hue='Class',
    alpha=0.5,
    palette={'Não Fraude': "steelblue", 'Fraude': "red"}
)

plt.title('Separação Espacial: V3 vs V14')
plt.show()

A projeção bidimensional no espaço formado por `V3` e `V14` revela separação espacial clara entre as classes, indicando que fraudes ocupam regiões específicas do espaço PCA, enquanto transações legítimas permanecem concentradas próximas à origem.

### **Análise Multivariada**

In [ ]:
sample_df = df_train.sample(8000, random_state=42)

fig = plt.figure(figsize=(6,5))
ax = fig.add_subplot(111, projection='3d')

colors = sample_df['Class'].map({0: '#1f77b4', 1: '#d62728'})

ax.scatter(
    sample_df['V17'],
    sample_df['V14'],
    sample_df['V3'],
    c=colors,
    alpha=0.5,
    s=15
)

ax.set_xlabel('V17')
ax.set_ylabel('V14')
ax.set_zlabel('V3')
ax.set_title('Separação 3D no Espaço PCA')

plt.show()


A projeção tridimensional das três variáveis com maior diferença entre classes `V17, V14` e `V3` evidencia a ocupação de uma região distinta por transações fraudulentas no espaço PCA. Como essas variáveis são ortogonais, a separação observada não decorre de colinearidade, mas de diferenças estruturais multivariadas entre os grupos.

A concentração das transações legítimas próxima à origem e o deslocamento das fraudes em múltiplas direções independentes sugerem que o problema apresenta separabilidade não linear, uma justificativa para a futura escolha de modelos baseados em árvores de decisão e boosting.

Embora a separação não seja perfeitamente linear, observa-se clara tendência de agrupamento, indicando que combinações dessas componentes ampliam a capacidade discriminativa.

### **Multicolinearidade**

In [ ]:
v_cols = [col for col in df_train.columns if col.startswith('V')]

corr_matrix = df_train[v_cols].corr()

plt.figure(figsize=(7,6))
sns.heatmap(corr_matrix, cmap='coolwarm', center=0)
plt.title('Correlação entre Componentes PCA (V1–V28)')
plt.show()


As variáveis `V1–V28` correspondem a componentes obtidas por transformação PCA originalmente vindas do dataset, portanto, ortogonais e não correlacionadas entre si.

A matriz de correlação confirma ausência de correlações lineares relevantes entre essas variáveis, descartando a presença de multicolinearidade no conjunto de atributos principais.

### **Conclusão**

A análise exploratória revelou um cenário com forte desbalanceamento de classes, em que apenas aproximadamente 0,17% das transações correspondem à classe fraudulenta. Esse contexto invalida o uso de métricas como acurácia e justifica a adoção da **ROC-AUC** como métrica principal de avaliação, por medir a capacidade do modelo de ordenar corretamente instâncias de maior risco.

As variáveis `V1–V28`, apresentaram ortogonalidade estrutural confirmada empiricamente pela matriz de correlação, descartando presença de multicolinearidade relevante. Essa característica favorece estabilidade em modelos lineares e reduz risco de redundância informacional.

A análise univariada evidenciou forte deslocamento estatístico entre as classes em componentes específicas do espaço PCA, destacando-se principalmente `V17, V14, V12, V10` e `V3`. Essas variáveis apresentaram diferenças médias expressivas entre classes, sugerindo elevada capacidade discriminatória.

Projeções bidimensionais e tridimensionais no subespaço formado pelas principais componentes evidenciaram separabilidade espacial clara entre transações legítimas e fraudulentas, indicando que fraudes ocupam regiões específicas do espaço transformado. Tal comportamento sugere que o problema apresenta estrutura discriminativa relevante que será explorada por modelos lineares e não lineares.

Por outro lado, as variáveis originais `Time` e `Amount` apresentaram alta sobreposição entre classes. Embora `Amount` demonstre leve deslocamento médio, seu efeito padronizado é pequeno, indicando poder discriminatório isolado limitado. A variável temporal mostrou variações locais na taxa de fraude, mas com forte sobreposição global.

De forma geral, a EDA sugere que a separação entre classes é majoritariamente capturada no espaço PCA, com baixa dependência de variáveis originais.

## **PRÉ-PROCESSAMENTO**

### **Drop**

In [ ]:
df_train = df_train.drop(columns=['id'])
df_test = df_test.drop(columns=['id'])

X = df_train.drop('Class', axis=1)
y = df_train['Class']

Como a coluna `id` é apenas um identificador e não contém informação preditiva, realizei o drop nos conjuntos dos dados.

Não há presença de missing values, nem multicolinearidade. Também não há necessidade de escalonamento, pois as variáveis `V1-V28` já vieram de PCA e estão padronizadas.

In [ ]:
v_cols = [col for col in X.columns if col.startswith('V')]
amount_col = ['Amount']
time_col = ['Time']

# Transformação log para Amount
log_transformer = FunctionTransformer(np.log1p, validate=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('log_amount', log_transformer, amount_col),
        ('passthrough', 'passthrough', v_cols + time_col)
    ]
)

A variável `Amount` apresenta forte assimetria positiva. Para reduzir a influência de valores extremos e aproximar a distribuição de uma forma mais estável numericamente, aplica-se transformação logarítmica via log1p.

A variável `Time` é mantida sem transformação adicional, uma vez que sua escala já é compatível com os demais atributos e não apresenta explosão de magnitude que comprometa o treinamento.

A utilização de `ColumnTransformer` garante que as transformações sejam aplicadas de forma controlada e reprodutível dentro de pipelines, evitando vazamento de informação (data leakage) em validação cruzada.

## **BIBLIOTECAS PARA MODELAGEM**

In [9]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import FunctionTransformer

from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier

## **MODELAGEM**

### **Logistic Regression**

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

pipeline_lr = Pipeline([
    ('preprocess', preprocessor),
    ('model', LogisticRegression(
        class_weight='balanced',
        max_iter=1000,
        random_state=42
    ))
])

auc_lr = cross_val_score(
    pipeline_lr,
    X,
    y,
    cv=skf,
    scoring='roc_auc',
    n_jobs=-1
)

print(f"Logistic Regression CV AUC: {auc_lr.mean():.4f} ± {auc_lr.std():.4f}")

In [ ]:
# Treina modelo final para interpretação
pipeline_lr.fit(X, y)

In [ ]:
feature_names = (
    amount_col + v_cols + time_col
)

coefs = pipeline_lr.named_steps['model'].coef_[0]

coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefs
}).sort_values(by='Coefficient', key=abs, ascending=False)

coef_df.head(10)


A análise dos coeficientes da **Regressão Logística** confirmou os achados da etapa exploratória. As componentes `V14, V4, V10` e `V12`, previamente identificadas como apresentando maior deslocamento estatístico entre as classes, emergiram como as variáveis de maior magnitude nos coeficientes do modelo.

In [ ]:
coef_df['abs_coef'] = np.abs(coef_df['Coefficient'])
coef_df['odds_ratio'] = np.exp(coef_df['Coefficient'])

coef_df.sort_values('abs_coef', ascending=False).head(10)


A análise dos *odds ratios* revelou que determinadas componentes latentes exercem influência substancial sobre a probabilidade de fraude. Em particular, a componente `V14` apresentou *odds ratio* de 0.245, indicando que variações nessa dimensão alteram drasticamente o risco estimado. De forma complementar, `V4` e `V22` apresentaram *odds ratios* superiores a 2, sugerindo direções latentes associadas ao aumento significativo da probabilidade de transações fraudulentas. Esses resultados corroboram os achados da etapa exploratória, evidenciando que a separação entre classes é concentrada em poucas dimensões do espaço PCA.

In [ ]:
# Probabilidades do modelo final treinado em todos os dados
y_proba_full = pipeline_lr.predict_proba(X)[:, 1]

fpr, tpr, thresholds = roc_curve(y, y_proba_full)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, label=f'Logistic (AUC = {roc_auc:.4f})')
plt.plot([0,1], [0,1], linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - Logistic Regression')
plt.legend()
plt.show()

A curva ROC do modelo final treinado com todo o dataset apresenta AUC de aproximadamente 0.98, consistente com o desempenho médio estimado via validação cruzada.

In [ ]:
precision, recall, thresholds_pr = precision_recall_curve(y, y_proba_full)

plt.figure(figsize=(6,5))
plt.plot(recall, precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - Logistic Regression')
plt.show()

A curva **Precision-Recall1** evidencia desempenho robusto em cenário de forte desbalanceamento. Observa-se que o modelo mantém níveis de precisão próximos ou superiores a 85% até aproximadamente 75% de recall, indicando elevada capacidade de identificação das fraudes mais evidentes com controle adequado de falsos positivos.

A partir de níveis de recall mais elevados (acima de ~80%), há queda acentuada na precisão, refletindo o trade-off esperado em problemas de detecção de anomalias, nos quais ampliar a cobertura implica aumento substancial de falsos positivos.

O comportamento da curva sugere que o modelo produz um ranking bem ordenado das transações por risco, permitindo ajuste estratégico do threshold conforme restrições operacionais.

### **Random Forest**

In [ ]:
rf = RandomForestClassifier(
    n_estimators=300,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

auc_rf = cross_val_score(
    rf,
    X,
    y,
    cv=skf,
    scoring='roc_auc',
    n_jobs=-1
)

print(f"Random Forest CV AUC: {auc_rf.mean():.4f} ± {auc_rf.std():.4f}")

O modelo **Random Forest** apresentou desempenho inferior à **Regressão Logística** (AUC média de 0.9658 ± 0.0149), sem ganho em capacidade discriminativa.

Esse resultado sugere que grande parte da separação entre as classes já é capturada por fronteiras aproximadamente lineares no espaço PCA. Como as componentes `V1–V28` representam combinações lineares previamente otimizadas para maximizar variância explicada, é plausível que o problema apresente alta separabilidade linear nesse subespaço.

Modelos baseados em árvores não demonstraram ganho adicional significativo, indicando que interações não lineares complexas podem não ser determinantes para este conjunto específico de dados.

Além da menor média, o Random Forest apresentou maior variabilidade entre folds, sugerindo menor estabilidade em comparação ao modelo linear.

### **XGBoost**

In [ ]:
scale_pos_weight = (y == 0).sum() / (y == 1).sum()

xgb_base = XGBClassifier(
    n_estimators=300,
    max_depth=4,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    n_jobs=1,          # ← importante evitar paralelismo aninhado
    eval_metric='auc'
)

auc_xgb = cross_val_score(
    xgb_base,
    X,
    y,
    cv=skf,
    scoring='roc_auc',
    n_jobs=-1
)

print(f"XGBoost Baseline CV AUC: {auc_xgb.mean():.4f} ± {auc_xgb.std():.4f}")

O modelo **XGBoost baseline** apresentou AUC média de 0.9829 ± 0.0100, superando ligeiramente a **Regressão Logística**. Esse resultado indica que, embora grande parte da separação entre as classes já seja capturada por fronteiras aproximadamente lineares no espaço PCA, existe ganho marginal associado à modelagem de interações não lineares.

A magnitude relativamente pequena dessa diferença aponta a predominância linear da estrutura discriminativa do problema, mas com contribuições adicionais capturadas por mecanismos de boosting e particionamento hierárquico do espaço de atributos.

A partir desse baseline, ajustes incrementais nos hiperparâmetros foram explorados para avaliar possíveis melhorias adicionais sob validação cruzada estratificada.

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

auc_scores = []
best_iters = []

for train_idx, val_idx in skf.split(X, y):

    X_tr, X_va = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_va = y.iloc[train_idx], y.iloc[val_idx]

    xgb_cv = XGBClassifier(
        n_estimators=5000,
        max_depth=6,
        learning_rate=0.02,
        subsample=0.8,
        colsample_bytree=0.8,
        scale_pos_weight=(sum(y_tr==0)/sum(y_tr==1)),
        min_child_weight=5,
        random_state=42,
        n_jobs=-1,
        eval_metric='auc',
        early_stopping_rounds=200
    )

    xgb_cv.fit(
        X_tr,
        y_tr,
        eval_set=[(X_va, y_va)],
        verbose=False
    )

    y_pred = xgb_cv.predict_proba(X_va)[:,1]
    auc_scores.append(roc_auc_score(y_va, y_pred))
    best_iters.append(xgb_cv.best_iteration)

print("CV AUC mean:", np.mean(auc_scores))
print("CV AUC std:", np.std(auc_scores))
print("Best iteration médio:", np.mean(best_iters))



A validação do modelo foi conduzida por meio de **StratifiedKFold** com 5 divisões, garantindo a preservação da proporção entre classes em cada fold — aspecto essencial em cenários de forte desbalanceamento. O parâmetro `shuffle=True` foi utilizado para reduzir possíveis efeitos de ordenação no dataset.

Em cada iteração, os dados foram particionados em subconjuntos de treinamento (`X_tr, y_tr`) e validação (`X_va, y_va`). O modelo **XGBoost** (XGBClassifier) foi configurado com os seguintes hiperparâmetros:

* `n_estimators=5000`: limite superior elevado para permitir
exploração adequada do espaço de hipóteses;

* `max_depth=6`: profundidade moderada para capturar interações não lineares entre componentes;

* `learning_rate=0.02`: taxa de aprendizado reduzida para promover aprendizado gradual e maior estabilidade;

* `subsample=0.8` e `colsample_bytree=0.8`: introdução de aleatoriedade controlada para reduzir variância e correlação entre árvores;

* `scale_pos_weight=(sum(y_tr==0)/sum(y_tr==1))`: compensação explícita para o desbalanceamento entre classes;

* `min_child_weight=5`: regularização adicional para evitar crescimento excessivo das árvores;

* `early_stopping_rounds=200`: interrupção automática do treinamento caso não houvesse melhoria na métrica de validação.

Para cada fold, foi calculada a ROC-AUC, e o número ótimo de árvores foi determinado via *early stopping*. A média da AUC nos cinco folds foi utilizada como estimativa robusta de desempenho em dados não vistos, enquanto a média das melhores iterações forneceu indicação do ponto de convergência do modelo.

A validação cruzada resultou em AUC média de 0.9872, superando tanto a **Regressão Logística** quanto o **XGBoost baseline**. Esse ganho consistente indica que, embora grande parte da separabilidade já esteja representada por estruturas aproximadamente lineares no espaço PCA, existem padrões residuais e interações não lineares exploráveis pelo mecanismo de boosting sequencial.

Observou-se que o número médio de iterações ótimas foi aproximadamente 559 árvores, significativamente inferior ao limite máximo configurado (5000). Esse comportamento evidencia que o *early stopping* atuou de maneira eficaz na prevenção de sobreajuste, sugerindo convergência estável antes do esgotamento da capacidade máxima do modelo.

A combinação de *learning rate* reduzida com maior número potencial de estimadores permitiu refinamento progressivo das fronteiras de decisão, capturando variações sutis no espaço transformado sem comprometer a robustez em validação cruzada.

In [ ]:
ratio_full = sum(y == 0) / sum(y == 1)

final_model = XGBClassifier(
    n_estimators = int(np.mean(best_iters)),
    max_depth=6,
    learning_rate=0.02,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=ratio_full,
    min_child_weight=7,
    random_state=42,
    n_jobs=-1,
    eval_metric='auc'
)

final_model.fit(X, y)


### **Matriz de confusão**

In [ ]:
# Único fold de validação para análise de erros
skf_err = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
train_idx, val_idx = list(skf_err.split(X, y))[0]

X_tr, X_va = X.iloc[train_idx], X.iloc[val_idx]
y_tr, y_va = y.iloc[train_idx], y.iloc[val_idx]

model_err = XGBClassifier(
    n_estimators=559,
    max_depth=6,
    learning_rate=0.02,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=ratio_full,
    min_child_weight=7,
    random_state=42,
    n_jobs=-1,
    eval_metric='auc'
)
model_err.fit(X_tr, y_tr)

# Threshold padrão 0.5
y_pred_class = (model_err.predict_proba(X_va)[:,1] >= 0.5).astype(int)

cm = confusion_matrix(y_va, y_pred_class)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Legítima', 'Fraude'])
disp.plot(cmap='Blues')
plt.title('Matriz de Confusão — XGBoost (threshold = 0.5)')
plt.show()

tn, fp, fn, tp = cm.ravel()
print(f"Verdadeiros Positivos (fraudes detectadas): {tp}")
print(f"Falsos Negativos (fraudes não detectadas): {fn}")
print(f"Falsos Positivos (legítimas classificadas como fraude): {fp}")
print(f"Verdadeiros Negativos: {tn}")
print(f"Recall de fraude: {tp/(tp+fn):.4f}")
print(f"Precisão de fraude: {tp/(tp+fp):.4f}")

O modelo identificou corretamente 65 das 78 transações fraudulentas presentes
no fold de validação, atingindo recall de 0.8333 e precisão de 0.7558.

Os 13 falsos negativos representam fraudes não interceptadas — o erro de maior
impacto em contextos reais, onde cada transação fraudulenta não detectada
implica prejuízo financeiro direto.

Os 21 falsos positivos correspondem a transações legítimas incorretamente
bloqueadas. Embora gerem atrito operacional, representam apenas 0.046% das
transações legítimas do fold, indicando baixo impacto sobre a experiência
do usuário.

O threshold de 0.5 é um ponto de partida conservador. Em cenários reais,
o limiar ótimo é definido com base no custo relativo entre os dois tipos de erro — informação que, neste dataset anonimizado, não está disponível.

### **BIBLIOTECAS PARA SHAP**

In [10]:
import shap

### **SHAP**

Como o **XGBoost** é um modelo baseado em **boosting** e possui múltiplas árvores de decisão complexas, é considerado estruturalmente mais complexo que modelos lineares. não deixando tão claro como as variáveis de entrada afetam a previsão final.

Para garantir que o modelo final seja explicável é preciso utilizar a técnica **SHapley Additive exPlanations (SHAP)**, que permite entender a **contribuição de cada variável** para a decisão do modelo, mostrando como o valor de cada feature impacta a previsão final de **fraude** ou **não fraude**, garantindo auditabilidade.

### **Summary Plot (Interpretabilidade Global)**

In [ ]:
# Amostra fixa para interpretabilidade
X_shap = X.sample(5000, random_state=42)

explainer = shap.TreeExplainer(final_model)
shap_values = explainer.shap_values(X_shap)

plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_shap, plot_type="dot")
plt.show()

O **SHAP Summary Plot** fornece uma decomposição global do impacto das variáveis no *log-odds* predito pelo modelo, permitindo compreender como o **XGBoost** utiliza o espaço transformado por PCA para separar as classes.

Observa-se que as componentes `V14, V4, V12` e `V10` concentram os maiores valores absolutos de **SHAP**, indicando maior contribuição marginal média na decisão do modelo. Existe uma relação direcional consistente: valores fortemente negativos de `V14, V12` e `V10` (azul no gráfico) deslocam o *log-odds* para a classe positiva (fraude), enquanto valores elevados de `V4` aumentam o risco predito.

A consistência entre magnitude dos coeficientes da **Regressão Logística** e a importância **SHAP** no **XGBoost** sugere que o ensemble está explorando padrões estruturais já identificados pelo modelo linear.

Essa convergência reforça a estabilidade interpretativa da solução e reduz o risco de decisões arbitrárias típicas de modelos não explicados.

### **Waterfall Plot (Interpretabilidade Local)**

In [ ]:
# Fraude real
fraud_idx = y[y == 1].index[0]
pos = X.index.get_loc(fraud_idx)

shap.waterfall_plot(shap.Explanation(
    values=explainer.shap_values(X.iloc[[pos]])[0],
    base_values=explainer.expected_value,
    data=X.iloc[pos],
    feature_names=X.columns
))

O **SHAP Waterfall Plot** ilustra a decomposição individual da predição para uma transação corretamente classificada como fraude. O valor base esperado do modelo (E[f(X)] = 1.725) foi deslocado até f(x) = 8.006 em *log-odds*, correspondendo a probabilidade predita superior a 99%.

A principal contribuição positiva foi proveniente de `V14` (+3.29), seguida por `V4` (+1.24) e `V10` (+1.16), evidenciando que o modelo atribuiu alto risco à combinação específica dessas componentes.

Simultaneamente, variáveis como `V8` e `V26` exerceram impacto negativo parcial, demonstrando que a decisão final resulta da soma balanceada de forças contribuintes e não de uma única variável dominante.

Essa decomposição aditiva torna a decisão auditável e rastreável, permitindo compreender quais componentes do espaço PCA foram determinantes para a classificação individual.

## **COMPARAÇÃO ESTRUTURADA ENTRE MODELOS**

Para avaliar de forma sistemática a contribuição incremental de complexidade modelística, foram comparados três paradigmas distintos de aprendizado supervisionado:

- **Modelo Linear (Regressão Logística)**
- **Ensemble por Bagging (Random Forest)**
- **Boosting Sequencial (XGBoost)**

A tabela abaixo resume o desempenho médio sob validação cruzada estratificada (5 folds):

| Modelo                     | AUC Média | Desvio Padrão | Observações Principais |
|----------------------------|-----------|---------------|------------------------|
| Regressão Logística        | 0.9805    | 0.0114        | Forte separabilidade linear no espaço PCA |
| Random Forest              | 0.9658    | 0.0149        | Maior variabilidade entre folds |
| XGBoost (Baseline)         | 0.9829    | 0.0100        | Pequeno ganho via interações não lineares |
| XGBoost (Tuned)            | 0.9872    | 0.0078        | Melhor desempenho e maior estabilidade |

### **Análise Comparativa**

A **Regressão Logística** já alcança desempenho elevado, indicando que grande parte da separação entre classes é aproximadamente linear no espaço PCA.

O **Random Forest** apresentou desempenho inferior e maior variabilidade entre folds, sugerindo que particionamentos independentes do espaço não exploram eficientemente a estrutura discriminativa do problema.

O **XGBoost baseline** mostrou ganho marginal sobre o modelo linear, evidenciando a presença de interações não lineares residuais.

Após ajuste fino com *learning rate* reduzido e *early stopping*, o **XGBoost** atingiu AUC média de **0.9872**, representando ganho absoluto de aproximadamente **0.0067 pontos de AUC** em relação à Regressão Logística.

Esse incremento, embora numericamente modesto, é consistente entre folds e estatisticamente estável, indicando extração adicional de sinal preditivo sem evidência de sobreajuste.

### **Interpretação Estrutural**

A comparação evidencia que:

- O problema possui **alta separabilidade linear estrutural**, herdada da transformação PCA.
- Modelos mais complexos oferecem ganhos incrementais, mas não transformacionais.
- O ganho obtido pelo boosting sugere existência de padrões não lineares sutis, exploráveis por aprendizado sequencial.
- A baixa variabilidade entre folds reforça robustez e estabilidade da solução final.

Essa análise fundamenta a escolha do **XGBoost** como modelo final, equilibrando desempenho preditivo, controle de overfitting e interpretabilidade via **SHAP**.

### **Conclusão**

A análise exploratória indicou que a separabilidade entre transações fraudulentas e legítimas está majoritariamente concentrada no espaço PCA, com forte deslocamento estatístico em componentes específicas.

Modelos lineares já capturam grande parte dessa estrutura discriminativa, atingindo AUC próxima de 0.98. A aplicação de boosting sequencial via **XGBoost** permitiu explorar interações residuais não lineares, elevando o desempenho para AUC média de 0.9872 sob validação cruzada estratificada.

O uso de *early stopping* e avaliação *out-of-fold* demonstrou convergência estável e ausência de evidências de sobreajuste estrutural.

A interpretabilidade foi assegurada por meio de **SHAP**, com explicações globais e locais consistentes com os coeficientes do modelo linear preliminar. Tal convergência sugere que o modelo final não opera como caixa-preta arbitrária, mas explora padrões estatísticos coerentes no espaço transformado.

O conjunto de resultados indica solução robusta, estável e auditável, adequada para cenários reais de detecção de fraude sob forte desbalanceamento de classes

## **SALVANDO RESULTADOS**

In [12]:
import joblib

In [ ]:
df_test = pd.read_csv('../data/test.csv')

test_ids = df_test['id']
X_test = df_test.drop(columns=['id'])

test_proba = final_model.predict_proba(X_test)[:,1]

submission = pd.DataFrame({
    'id': test_ids,
    'target': test_proba
})

submission.to_csv('../submission_final_ligia.csv', index=False)

final_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('model', final_model)
])

joblib.dump(final_pipeline, '../models/modelo_xgboost_ligia.pkl')

In [15]:
!pip freeze > requirements.txt